In [4]:
df_ori = pd.read_excel(r'D:\ODOC\jupyter_notebook\제품데이터_FULL\제품_성분_정리.xlsx')
df = df_ori

In [5]:
# True값이 아예 없음 -> 삭제
df = df.drop(['속눈썹컨디셔닝제','염모제','항료','흡착제'], axis=1)

# 상관계수 0.01미만 -> 삭제
drop_columns = ['감미제', '스킨', '피막형성제', '분사제', '활성제(세정제)', 
        '헤어스트레이트너용제', '퍼머넌트웨이브용제','용제', '점착제',
       '변색방지제', '세포용해제', '연마제', '계면활성제', '피부보호제', 
       '스킨컨디셔닝제', '수렴제', '금속이온봉쇄제', '착향제',
               ]
df = df.drop(drop_columns, axis=1)

In [6]:
# 널값 처리
df.loc[df.price.isna(), 'price'] = df.loc[df.price.isna(),'sale_price']
df.review_star = df.review_star.fillna(0)
df.category_modify = df.category_modify.fillna('기타')
df = df.fillna(0)

#세일가 -> 세일 %로 변경
df['sale_percentage'] = ((df['price'] - df['sale_price'])/df['price'])*100

In [7]:
def price_level(col):
    cat = df[df.category_modify==col]
    per20_cnt = round(len(cat)*0.2)
    
    sort_price = cat.sort_values('price')

    low0 = sort_price.iloc[:per20_cnt].index
    low1 = sort_price[per20_cnt:(per20_cnt*2)].index
    low2 = sort_price[(per20_cnt*2):(per20_cnt*3)].index
    low3 = sort_price[(per20_cnt*3):(per20_cnt*4)].index
    low4 = sort_price[(per20_cnt*4):].index
    return low0,low1,low2,low3,low4

In [1]:
# df.category_modify.unique()

In [9]:
# 가격등급 칼럼에 아예 카테고리 정보도 넣자(ex. 에센스-고가)
# 썬제품에서는 2만원이 고가인데, 토너에서는 10만원이 고가고, 이런 상황x
df['level'] = None

for col in df.category_modify.unique():
    low0,low1,low2,low3,low4 = price_level(col)
    df.loc[low0, 'level'] = col+'0'
    df.loc[low1, 'level'] = col+'1'
    df.loc[low2, 'level'] = col+'2'
    df.loc[low3, 'level'] = col+'3'
    df.loc[low4, 'level'] = col+'4'
    
# 결론 -> 그다지 좋은 결과x
# 데이터 수가 적어서 지나치게 세분화되는 느낌

In [149]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np

#원핫 인코딩: 카테고리
dummy_df = pd.get_dummies(df.category_modify)
new_df = pd.concat([df, dummy_df], axis=1)
new_df.drop('category_modify',axis=1, inplace=True)

#원핫 인코딩: 가격등급
dummy_df = pd.get_dummies(new_df.level)
new_df = pd.concat([new_df, dummy_df], axis=1)
new_df.drop('level',axis=1, inplace=True)

# #원핫 인코딩: 브랜드명(983개 칼럼)
# dummy_df_brand = pd.get_dummies(new_df.brand_name)
# new_df = pd.concat([new_df, dummy_df_brand], axis=1)
# new_df.drop('brand_name', axis=1, inplace=True)

# 레이블 인코딩: 브랜드명
label_encoder = LabelEncoder()
new_df.brand_name = label_encoder.fit_transform(new_df.brand_name)

#로그변환: 가격
new_df['log_price'] = np.log(new_df['price'] + 1)

#스케일러: 타겟값은 스케일링 하면 값 복구가 힘듬
# scaler = MinMaxScaler()
# new_df['price_minmax'] = scaler.fit_transform(new_df['price'].values.reshape(-1,1))
# new_df['price_minmax'] = new_df['price_minmax'].apply(lambda x: x*100)

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
corr_matrix = new_df.corr()
sort_df = corr_matrix['price'].abs().sort_values(ascending=False)

sort_df[sort_df<0.01].index
# sort_df[:20]

In [710]:
# 브랜드별로 레벨별 개수 세기 ->
# 저가/중저가/중가/고가/최고가 별 브랜드 나눠보는 것도 좋을 듯
#(현재는 칼럼이 너무 많고, 영향도 낮음)

# level_counts_by_brand = df.groupby(['brand_name', 'level']).size().unstack(fill_value=0)
# print(level_counts_by_brand)

level        0  1  2  3  4
brand_name                
3DAMO        0  5  0  0  0
40패덤즈        0  2  0  1  2
A3F on       0  0  0  1  0
AU8          0  0  0  2  1
AYU25        1  0  1  2  1
...         .. .. .. .. ..
후르디아         5  1  3  1  0
휘게           2  1  1  1  0
히든랩          0  4  0  3  0
히말라야        17  4  1  0  0
힐시드          3  0  0  0  0

[983 rows x 5 columns]


In [154]:
from sklearn.model_selection import train_test_split
target = 'log_price'
x = new_df.drop(['product_code','price','sale_price',
                target],axis=1)
y = new_df[target]

x_train, x_test, y_train, y_test = train_test_split(x,y,
                                test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,
                                test_size=0.3, random_state=42)

In [155]:
print(len(x_train), len(x_val), len(x_test))
print(len(y_train), len(y_val), len(y_test))

3466 1486 1239
3466 1486 1239


In [156]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
def model_metrics(model_pred, y):
    model_mse = mean_squared_error(y, model_pred)
    model_mae = mean_absolute_error(y, model_pred)
    model_r2 = r2_score(y, model_pred)
    model_rmse = np.sqrt(model_mse)
    model_mpe = np.mean((y-model_pred)/y)*100
    
    return model_mse,model_mae,model_r2,model_rmse,model_mpe

In [159]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
lr = LinearRegression()
rd = Ridge(alpha=0.5, max_iter=100)
lass = Lasso(alpha=0.5, max_iter=100)

lr.fit(x_train, y_train)
rd.fit(x_train, y_train)
lass.fit(x_train, y_train)

lr_pred = lr.predict(x_val)
rd_pred = rd.predict(x_val)
lass_pred = lass.predict(x_val)

lr_metrics = model_metrics(lr_pred, y_val)
rd_metrics = model_metrics(rd_pred, y_val)
lass_metrics = model_metrics(lass_pred, y_val)

In [125]:
print(lr_metrics)
print(rd_metrics)
print(lass_metrics)

(0.10458128807452377, 0.2151357365523238, 0.8940123167283406, 0.3233903030001422, -0.03679704441619799)
(0.10453233458777081, 0.21462236278263555, 0.8940619285350466, 0.3233146062085207, -0.05240404072496963)
(0.9697193530119516, 0.7159806365697522, 0.01723999061677073, 0.9847432929509861, -0.8218305414987597)


In [126]:
pred = pd.DataFrame({'price':y_val,'lr_pred':lr_pred,
                     'rd_pred':rd_pred, 'lass_pred':lass_pred})
idx = pred.index
pred['price'] = df.price[idx].astype(int)

pred['lr_pred_ori'] = round(np.exp(pred.lr_pred), -2).astype(int)
pred['rd_pred_ori'] = round(np.exp(pred.rd_pred), -2).astype(int)
pred['lass_pred_ori'] = round(np.exp(pred.lass_pred), -2).astype(int)

pred[['price','lr_pred_ori','rd_pred_ori','lass_pred_ori']][20:30]

,price,lr_pred_ori,rd_pred_ori,lass_pred_ori
2550,11500,10300,10300,24900
3322,49000,55600,55600,24900
2352,32000,32600,32400,24900
5525,35000,42500,42300,24900
3784,70000,56400,56000,26200
52,98000,144300,142900,24900
3012,25500,23900,23900,24900
3196,8000,4900,4900,26700
2513,37000,39300,39500,25300
4598,14900,9700,9800,24900


In [127]:
#라쏘에서 사용된 모델 특성(피처) 확인
lasso_coef = lass.coef_
for feature, coef in zip(x.columns, lasso_coef):
    if coef != 0:
        print(f"{feature}: {coef}")

brand_name: 3.3018818194468924e-06
review_num: 4.6796731667760996e-05
sale_percentage: 0.01002949286455978


In [128]:
#앙상블 모델(랜덤포레스트 2개)
from sklearn.ensemble import RandomForestRegressor
rf1 = RandomForestRegressor(n_estimators=200, 
                            min_samples_split=2)
rf2 = RandomForestRegressor(n_estimators=200, 
                            min_samples_split=2)

rf1.fit(x_train, y_train)
rf2.fit(x_train, y_train)

pred1 = rf1.predict(x_val)
pred2 = rf2.predict(x_val)

ensemble_pred = (pred1 + pred2) / 2

In [176]:
# print(model_metrics(pred['lr_pred_ori'], pred['price']))

rf_ans_metrics = model_metrics(ensemble_pred, y_val)
rf_ans_metrics

(0.09424398392548969,
 0.193710279916369,
 0.9044886355632158,
 0.306991830388839,
 -0.005243186816009736)

In [131]:
#로그변환 되돌리기
pred['rf_ans_pred_ori'] = np.round(np.exp(ensemble_pred),-2).astype(int)

In [132]:
pred[['price','lr_pred_ori','rd_pred_ori',
      'rf_ans_pred_ori']][1400:1410]

,price,lr_pred_ori,rd_pred_ori,rf_ans_pred_ori
2976,55000,40300,40100,44400
6006,9400,12800,12900,12400
2171,29700,31000,31100,29600
6188,24000,21800,21800,22800
1607,26790,22600,22600,24800
3884,44000,58800,58700,57900
572,17600,17600,17600,18600
4747,110200,121300,120300,91300
10,5100,10500,10600,10600
5235,20000,19000,19000,20900


In [178]:
model_metrics(pred['lr_pred_ori'], pred['price'])

(76728936.44683714,
 13006.890982503364,
 0.980006445268236,
 8759.505490998743,
 -5.1953331810142656)

In [723]:
#그리드 서치
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
param_grid = {
    'n_estimators': [200, 250, 300],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2]
}

grid_search = GridSearchCV(estimator=model, param_grid=
            param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(x_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {'max_depth': 15, 'min_samples_split': 2, 'n_estimators': 200}
Best Score: -0.09560620553237734


In [464]:
# !pip install xgboost

In [134]:
import xgboost as xgb
import lightgbm as lgb

xgb_model = xgb.XGBRegressor()
lgb_model = lgb.LGBMRegressor(learning_rate=0.15,
                           n_estimators=300)

xgb_model.fit(x_train, y_train)
lgb_model.fit(x_train, y_train)

xgb_pred = xgb_model.predict(x_val)
lgb_pred = lgb_model.predict(x_val)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001157 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 797
[LightGBM] [Info] Number of data points in the train set: 3466, number of used features: 92
[LightGBM] [Info] Start training from score 10.171039


In [135]:
model_metrics(xgb_pred, y_val)

(0.09410557363552145,
 0.19740135057952327,
 0.9046289071762813,
 0.3067663176352995,
 -0.028876911377107593)

In [136]:
model_metrics(lgb_pred, y_val)

(0.09820119899641329,
 0.20993087569889315,
 0.900478204392431,
 0.31337070538966033,
 0.012026328506030906)

In [137]:
#로그변환 되돌리기
pred['lgb_pred_ori'] = np.round(np.exp(lgb_pred),-2).astype(int)
pred['xgb_pred_ori'] = np.round(np.exp(xgb_pred),-2).astype(int)

In [138]:
pred[['price','lr_pred_ori','rd_pred_ori',
      'rf_ans_pred_ori',
     'lgb_pred_ori','xgb_pred_ori']][100:105]

,price,lr_pred_ori,rd_pred_ori,rf_ans_pred_ori,lgb_pred_ori,xgb_pred_ori
1853,55000,41900,42000,43200,46800,41200
3745,52000,54700,55100,44500,40000,43100
3548,78000,132400,132000,137300,116400,161900
216,55000,73900,73200,75500,75100,85800
829,8000,5400,5400,5500,4100,5300


In [179]:
print(model_metrics(pred['xgb_pred_ori'], pred['price']), '\n')
print(model_metrics(pred['lgb_pred_ori'], pred['price']))

(84269005.44279946, 12142.812920592194, 0.978041700425767, 9179.815109401685, -4.789623263639326) 

(91833638.04576044, 11729.152086137281, 0.9760705549495388, 9582.986906270948, -4.681987184008457)


In [730]:
#테스트 셋에도 비슷하게 나옴
xgb_test = xgb_model.predict(x_test)
lgb_test = lgb_model.predict(x_test)
print(model_metrics(xgb_test, y_test))
print(model_metrics(lgb_test, y_test))

(0.08296200643168865, 0.19941697789049104, 0.9115023276078792, 0.28803125946967745, 0.07768903659500252)
(0.08295939137904715, 0.2030956793171262, 0.9115051171507303, 0.2880267199046768, 0.06228668108820171)
